In [75]:
%load_ext autoreload
%autoreload 2
import itertools
import tempfile
import pipelines
import json
import time
import os
import glob
import datetime
from collections import defaultdict, OrderedDict
from wikidata_parser import WikiDataParser, WikiDataInheritanceGraph
import wikitextparser as wtp
import psycopg2
import dateparser
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import shelve
db_path = "data/wikilanguage.db"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
with pipelines._buffered_stream("wikis-2019-11-17/latest-all.json.bz2") as f:
    for i, entry in enumerate(WikiDataParser.parse_dump(f)):
        if i > 100:
            break

In [8]:
t = shelve.open("temp.shelf")

In [3]:
db = WikilanguageDB.con_fast_ro(db_path)
with db.con as cur:
    wikis = set(e[0] for e in cur.execute("SELECT DISTINCT(wiki) FROM articles").fetchall())

In [6]:
with db.con as cur:
    nice_order = sorted(wikis, key=lambda x: "!" if x == "enwiki" else x)
    exploder = ",\n\t".join(f"""
        MAX(
            CASE(articles.wiki)
            WHEN '{wiki}' THEN articles.title
            ELSE NULL
            END
        ) AS {wiki}_title, 
        MAX(
            CASE(articles.wiki)
            WHEN '{wiki}' THEN articles.pagerank
            ELSE NULL
            END
        ) AS {wiki}_pagerank
    """.strip() for wiki in nice_order)
    
    sql = f"""
        SELECT
        concepts.concept_id,
        {exploder},
        GROUP_CONCAT(
            concept_instance_of.instance_of_concept_id
        ) concept_instances,
        GROUP_CONCAT(
            CASE(concept_instance_of.is_direct_instance)
               WHEN 1 THEN concept_instance_of.instance_of_concept_id
               ELSE NULL
            END
        ) AS direct_instances
        FROM concepts 
        LEFT OUTER JOIN articles ON (
            articles.concept_id=concepts.concept_id
        )
        LEFT OUTER JOIN concept_instance_of ON (
            concepts.concept_id=concept_instance_of.concept_id
        )
        
        GROUP BY concepts.concept_id 
        ORDER BY enwiki_pagerank DESC 
    """
    print(sql)


        SELECT
        concepts.concept_id,
        MAX(
            CASE(articles.wiki)
            WHEN 'enwiki' THEN articles.title
            ELSE NULL
            END
        ) AS enwiki_title, 
        MAX(
            CASE(articles.wiki)
            WHEN 'enwiki' THEN articles.pagerank
            ELSE NULL
            END
        ) AS enwiki_pagerank,
	MAX(
            CASE(articles.wiki)
            WHEN 'arwiki' THEN articles.title
            ELSE NULL
            END
        ) AS arwiki_title, 
        MAX(
            CASE(articles.wiki)
            WHEN 'arwiki' THEN articles.pagerank
            ELSE NULL
            END
        ) AS arwiki_pagerank,
	MAX(
            CASE(articles.wiki)
            WHEN 'dewiki' THEN articles.title
            ELSE NULL
            END
        ) AS dewiki_title, 
        MAX(
            CASE(articles.wiki)
            WHEN 'dewiki' THEN articles.pagerank
            ELSE NULL
            END
        ) AS dewiki_pagerank,
	MAX(
      